# Скачиваем и устанавливаем всё необходимое

Удаляем ненужный sample_data

In [80]:
!rm -rf sample_data

Клонируем наш репозиторий

In [70]:
!git clone https://github.com/HSE-projects/logical_reasoning

Cloning into 'logical_reasoning'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 34 (delta 4), reused 26 (delta 4), pack-reused 4
Unpacking objects: 100% (34/34), done.
/content/logical_reasoning


Удаляем torchtext так как в модели хотят версию торча 1.6, а она не поддерживается с новым torchtext, torchvision, ...

In [59]:
!pip uninstall torchtext -y

Устанавливаем всё необходимое

In [60]:
!pip install -r logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/requirements.txt
!pip install datasets
!pip install gdown

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'Self_Explaining_Structures_Improve_NLP_Models/requirements.txt'


Скачиваем оптимальные параметры модели

In [71]:
!gdown https://drive.google.com/uc?id=1AHMGgYFqcr3NmA-py6AMsO83w8FeNSJn  # SNLI Large
#!gdown https://drive.google.com/uc?id=1bJVYekaOVJjI2woxqaqKMhWP0HLau105  # SNLI Base

Downloading...
From: https://drive.google.com/uc?id=1AHMGgYFqcr3NmA-py6AMsO83w8FeNSJn
To: /content/logical_reasoning/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt
100% 4.31G/4.31G [01:11<00:00, 60.6MB/s]


Скачиваем roberta-large. Для этого надо установить [git-lfs](https://git-lfs.github.com)

In [73]:
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/roberta-large

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 85 not upgraded.
Updated git hooks.
Git LFS initialized.
Cloning into 'roberta-large'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 65 (delta 29), reused 0 (delta 0)
Unpacking objects: 100% (65/65), done.
tcmalloc: large alloc 1471086592 bytes == 0x56487d132000 @  0x7f732435c2a4 0x564840d6eb9f 0x564840d4bcdb 0x564840d00e83 0x564840ca518a 0x564840ca5646 0x564840cc23b1 0x564840cc2949 0x564840cc2e73 0x564840d67892 0x564840c09112 0x564840befa15 0x564840bf06d5 0x564840bef6da 0x7f73236a2bf7 0x564840bef72a
tcmalloc: large alloc 1471086592 bytes == 0x56487d132000 @  0x7f732435c2a4 0x564840d6eb9f 0x564840d4bcdb 0x564840d00e83 0x564840ca518a 0x564840ca5646 0x564840cc23b1 0x564840cc2949 0x564840cc2e73 0x5

# Тестируем модель

## Тестируем модель на SNLI

Скачиваем и распаковываем SNLI. Также удаляем ненужные файлы.

In [72]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
!unzip snli_1.0.zip
!rm -rf __MACOSX
!rm snli_1.0.zip

--2021-12-23 12:34:29--  https://nlp.stanford.edu/projects/snli/snli_1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94550081 (90M) [application/zip]
Saving to: ‘snli_1.0.zip’

snli_1.0.zip        100%[===================>]  90.17M  4.98MB/s    in 20s     

2021-12-23 12:34:50 (4.45 MB/s) - ‘snli_1.0.zip’ saved [94550081/94550081]

Archive:  snli_1.0.zip
   creating: snli_1.0/
  inflating: snli_1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/snli_1.0/
  inflating: __MACOSX/snli_1.0/._.DS_Store  
 extracting: snli_1.0/Icon           
  inflating: __MACOSX/snli_1.0/._Icon  
  inflating: snli_1.0/README.txt     
  inflating: __MACOSX/snli_1.0/._README.txt  
  inflating: snli_1.0/snli_1.0_dev.jsonl  
  inflating: snli_1.0/snli_1.0_dev.txt  
  inflating: snli_1.0/snli_1.0_test.jsonl  
  inflating: snli_1.0/snli_1.0_

Меняем конфиг, чтобы был `num_labels` = 3

In [97]:
import json
config = json.load(open('roberta-large/config.json'))
config['num_labels'] = 3
with open('roberta-large/config.json', 'w') as f:
    json.dump(config, f, indent=4)

{'architectures': ['RobertaForMaskedLM'], 'attention_probs_dropout_prob': 0.1, 'bos_token_id': 0, 'eos_token_id': 2, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 1024, 'initializer_range': 0.02, 'intermediate_size': 4096, 'layer_norm_eps': 1e-05, 'max_position_embeddings': 514, 'model_type': 'roberta', 'num_attention_heads': 16, 'num_hidden_layers': 24, 'pad_token_id': 1, 'type_vocab_size': 1, 'vocab_size': 50265, 'num_labels': 3}


Запускаем тестилку:

In [99]:
!mkdir /content/snli_result
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-large \
--data_dir /content/snli_1.0 \
--task snli \
--checkpoint_path '/content/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt' \
--save_path /content/snli_result/ \
--gpus=0, \
--mode eval
%cd /content

mkdir: cannot create directory ‘/content/snli_result’: File exists
/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1
Testing: 100% 983/983 [04:43<00:00,  4.26it/s]tensor(-0.7510, device='cuda:0') tensor(0.9221, device='cuda:0')
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The testing_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar

In [119]:
ls

'epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt'   SICK.zip
 logical_reasoning/                                      snli_1.0/
 roberta-large/                                          snli_result/


## Тестируем модель на SICK

In [153]:
!mkdir /content/sick_result
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-large \
--task sick \
--checkpoint_path '/content/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt' \
--save_path /content/sick_result/ \
--gpus=0, \
--mode eval
%cd /content

mkdir: cannot create directory ‘/content/sick_result’: File exists
/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1
Using custom data configuration default
Reusing dataset sick (/root/.cache/huggingface/datasets/sick/default/0.0.0/c6b3b0b44eb84b134851396d6d464e5cb8f026960519d640e087fe33472626db)
100% 3/3 [00:00<00:00, 296.66it/s]
Testing: 100% 491/491 [02:03<00:00,  4.32it/s]tensor(0.9206, device='cuda:0') tensor(0.6128, device='cuda:0')
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The testing_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning

## Тестируем модель на ANLI

In [174]:
!mkdir /content/anli_result
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-large \
--task anli \
--checkpoint_path '/content/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt' \
--save_path /content/anli_result/ \
--gpus=0, \
--mode eval
%cd /content

mkdir: cannot create directory ‘/content/anli_result’: File exists
/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1
Reusing dataset anli (/root/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)
100% 9/9 [00:00<00:00, 33.45it/s]
Testing: 100% 320/320 [05:18<00:00,  1.07s/it]tensor(0.9820, device='cuda:0') tensor(0.3278, device='cuda:0')
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The testing_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecate

# Итог


| Датасет | test_acc | test_loss | val_loss |
|---------|----------|-----------|----------|
| SNLI    | 0.9221   | -0.7510   | -0.7510  |
| SICK    | 0.6128   | 0.9206    | 0.9206   |
| ANLI    | 0.3278   | 0.9820    | 0.9820   |